In [1]:
import pandas as pd
import numpy as np
import json
import itertools
from apyori import apriori  

In [2]:
df = pd.read_csv("../data/location1_cleaned.csv")
del df['product_sku']
del df['display_description']
del df['location_name']
df_prods = pd.read_csv("../data_engineering/products_clean.csv")
df_prods_idx = df_prods.copy().set_index('id')

def get_name(id):
    return df_prods_idx.loc[id,'name']

def get_cat(id):
    return df_prods_idx.loc[id,'category']

def get_name_cat(id):
    return (df_prods_idx.loc[id,'name'],df_prods_idx.loc[id,'category'])

In [3]:
#Categories:
cats = list(set(list(df_prods['category'])))
cats

['Dessert', 'Alcoholic Beverage', 'Side', 'Entree', 'Beverage', 'Kids']

In [5]:
# remove kids
#
# Here we are doing the following: We remove the kids line items from orders. 
# Back over with the kids recommendations we followed a different strategy.
# It's not clear what's best.
#
df = df[ df['item_category_type'] != 'Kids' ]

# also throw out all alcoholic beverages
df = df[ df['item_category_type'] != 'Alcoholic Beverage' ]

# get unique list of order ids
order_ids = list(set(list(df['order_id'])))
print(len(order_ids))

13982


In [6]:
# Here we build the 2D orders array that will be fed into apyori
#
# The idea is the following: for each order, we consider all possible combinations
# of entree,side,beverage,dessert to handle the fact that often more than one of
# these categories is purchased. Based on the data, it's impossible to discern
# what belongs together, so we need to account for all possiblities. 
# Of course, this will reduce the predictive power of the outcome.
#

orders = []

# loop over all orders that fit our criteria
for i in range(len(order_ids)):
    
    # get all items for each order
    dd = (df.loc[ df['order_id'] == order_ids[i]]).copy()
    # drop duplicate items
    dd.drop_duplicates(subset='items_id',inplace=True)

    entree = list( dd.loc[ dd['item_category_type'] == 'Entree', 'items_id']  )
    nentree = len(entree)    
    dessert = list(dd.loc[ dd['item_category_type'] == 'Dessert', 'items_id'])
    ndessert = len(dessert)
    bev = list(dd.loc[ dd['item_category_type'] == 'Beverage','items_id'])
    nbev = len(bev)
    side = list(dd.loc[ dd['item_category_type'] == 'Side', 'items_id'])
    nside = len(side)
    
    #if i == 857:
    #print(f"{i} nk: {entree} ns: {nside} nb: {nbev} nd: {ndessert}")

    #if i == 237:
    #    print(dd)
    
    # loop over all the orders stuff
    for j in range(nentree):
        this_order = []
        if nside == 0 and nbev == 0 and ndessert == 0:
            this_order.append( [ entree[j] ] )
            continue

        if nside > 0:
            if nbev == 0 and ndessert == 0:
                xx = list(itertools.combinations(side,1))
            if nbev > 0 and ndessert == 0:
                xx = list(itertools.product(side,bev))
            if nbev == 0 and ndessert > 0:
                xx = list(itertools.product(side,dessert))
            if nbev > 0 and ndessert > 0:
                xx = list(itertools.product(side,dessert,bev))
        else:
            if nbev == 0 and ndessert == 0:
               assert(False) # this should never be reached
            if nbev > 0 and ndessert == 0:
                xx = list(itertools.combinations(bev,1))
            if nbev == 0 and ndessert > 0:
                xx = list(itertools.combinations(dessert,1))
            if nbev > 0 and ndessert > 0:
                xx = list(itertools.product(bev,dessert))
            
        for k in range(len(xx)):
            temp_order = []
            temp_order.append(entree[j])
            for l in range(len(xx[k])):
                temp_order.append(xx[k][l])
            this_order.append(temp_order)

    print(this_order)
    orders += this_order
    

[[3036]]
[[3036]]
[[3037, 1858]]
[[2716, 1813, 2245, 2218]]
[[2238, 1858, 2208]]
[[2716, 1813, 1858]]
[[3265]]
[[3285, 1858]]
[[2716, 1858]]
[[2210, 2218]]
[[2210, 2218]]
[[2210, 2218]]
[[3038, 2218], [3038, 1852], [3038, 1858]]
[[3038, 2218], [3038, 1852], [3038, 1858]]
[[3288]]
[[3036, 1858]]
[[2237]]
[[2237]]
[[2237]]
[[3038, 2251], [3038, 1856]]
[[2238]]
[[3285, 3289]]
[[4775, 1858]]
[[3038]]
[[3038, 1858]]
[[2238, 2251]]
[[2238, 2251]]
[[2237, 1858], [2237, 1855]]
[[3288, 2218]]
[[2210]]
[[2210]]
[[2210]]
[[2307]]
[[2237, 1813]]
[[2210]]
[[2210]]
[[3037, 2218], [3037, 1858]]
[[3265]]
[[3265]]
[[2716, 1858]]
[[2716, 1858]]
[[2716, 1858]]
[[3288, 1858]]
[[2237, 1852], [2237, 1858]]
[[2237, 1852], [2237, 1858]]
[[2210, 1858]]
[[3037, 1858]]
[[2210, 2218]]
[[2238]]
[[2238, 1858]]
[[2210, 1858]]
[[2210, 1858]]
[[2210, 1858]]
[[3288]]
[[2716, 1858], [2716, 2218]]
[[2716, 1858], [2716, 2218]]
[[3265]]
[[2210, 1858]]
[[3038, 2218]]
[[3265, 1858]]
[[3265, 1858]]
[[2238]]
[[3288]]
[[3265, 1

[[2237, 2222]]
[[2210, 1858]]
[[2237, 1858]]
[[3370, 1858]]
[[3285]]
[[3285]]
[[2210]]
[[3265]]
[[2210, 1858]]
[[3036, 1858]]
[[3288, 2218]]
[[3285, 1858]]
[[3285, 1858]]
[[3370]]
[[3038, 1858]]
[[3038]]
[[3285, 1858]]
[[3285]]
[[3288, 3289, 1858]]
[[2238]]
[[3037, 1855]]
[[3038, 3289, 1858]]
[[2237, 2218], [2237, 1852]]
[[3370]]
[[3285]]
[[2210, 1858]]
[[3389, 1858]]
[[3389, 1858]]
[[2237]]
[[3389]]
[[3285, 1852], [3285, 1858]]
[[3389]]
[[3038, 3411]]
[[3038, 3411]]
[[3037, 1858]]
[[2236]]
[[2307]]
[[2307, 2218]]
[[3288]]
[[3288]]
[[3285, 1858]]
[[2238, 1852], [2238, 1858]]
[[2716, 1858], [2716, 1852]]
[[2238]]
[[2307, 1852], [2307, 2218]]
[[3265, 1858]]
[[2716, 2218], [2716, 1858], [2716, 1852]]
[[2237, 1858]]
[[2237, 1858]]
[[2237, 1852]]
[[2307]]
[[2716]]
[[2210, 1858]]
[[2237, 1852]]
[[2238, 1858], [2238, 2218]]
[[2238, 1858, 3411]]
[[3288]]
[[2237, 1858], [2237, 3069]]
[[2210, 1852], [2210, 2218], [2210, 1858]]
[[3037]]
[[2307, 1858]]
[[3285]]
[[3288, 2221]]
[[2237, 1858]]
[[2237

[[2238, 2251]]
[[2236, 1852]]
[[3288, 1858]]
[[2237, 2218]]
[[3038]]
[[2210]]
[[2307]]
[[2716, 1858]]
[[3038]]
[[2716, 1858]]
[[2716, 1858]]
[[3038]]
[[2716, 1858]]
[[2236]]
[[2236]]
[[3265]]
[[3038, 1852]]
[[2238, 2218]]
[[3038, 2218]]
[[3037, 1858]]
[[2716, 1858]]
[[3038]]
[[2238, 1858]]
[[3038, 1858]]
[[3288, 2218]]
[[3389, 2245]]
[[3288, 1858]]
[[2238]]
[[3288, 2208]]
[[2210, 1852]]
[[3288, 1852], [3288, 1858]]
[[2210, 1852, 3411]]
[[2210, 2251]]
[[3285, 1858]]
[[3038, 1858], [3038, 2218]]
[[2716]]
[[2716, 1858]]
[[3288, 1858]]
[[2210, 1855], [2210, 1858]]
[[2716, 1858]]
[[2238, 1858], [2238, 1852]]
[[2307, 1858]]
[[2238]]
[[2307, 1858]]
[[2716, 1858]]
[[2238, 2218], [2238, 1858]]
[[2210, 1858]]
[[3389, 1858]]
[[2210, 2218], [2210, 1858]]
[[3265, 2218, 3411]]
[[2237, 2218]]
[[3288]]
[[2716, 1858]]
[[2716, 1858]]
[[2307, 1858, 3411]]
[[2238]]
[[2238]]
[[2236, 1858]]
[[2210, 1858]]
[[2307, 1813]]
[[2210, 1858]]
[[2716, 2218, 3411], [2716, 2218, 2245], [2716, 2218, 2208]]
[[2716, 2218

[[2307]]
[[2307, 1858, 3893]]
[[2307, 1858, 3893]]
[[2716, 1852]]
[[3288, 1858]]
[[2236, 1852]]
[[2210, 1858]]
[[2210, 1858]]
[[2238, 3893]]
[[3265, 1858]]
[[2238, 1858], [2238, 2251]]
[[3038, 1858], [3038, 1852]]
[[3036]]
[[3288, 1858]]
[[2210, 1858]]
[[2238, 1852], [2238, 1858]]
[[3265]]
[[2716]]
[[2210, 1858]]
[[2210, 1858]]
[[2716, 1858]]
[[2210, 1858]]
[[2307, 1858]]
[[3288, 2251]]
[[2716, 1852, 2245], [2716, 1852, 3411], [2716, 1858, 2245], [2716, 1858, 3411]]
[[2716, 1858]]
[[2716, 1852]]
[[2210, 2245]]
[[2307, 1813, 2245, 1858], [2307, 1813, 3411, 1858]]
[[2210, 1858]]
[[3037, 1858]]
[[3037, 1858]]
[[2237, 1858], [2237, 1852]]
[[2237, 1858], [2237, 1852]]
[[2238]]
[[2238]]
[[3265, 1852]]
[[2307, 1858]]
[[3265, 2218, 3411]]
[[3288]]
[[2210, 1813, 1852]]
[[2237]]
[[3265, 1852], [3265, 2218]]
[[3288, 2218]]
[[2307, 1855], [2307, 1858]]
[[3288, 1858]]
[[2210, 1852]]
[[3036]]
[[2236]]
[[2236]]
[[2238, 1858]]
[[2210, 1858]]
[[2238, 1858]]
[[2210, 1852]]
[[2716, 1858]]
[[3288, 1852], 

[[2716, 2207]]
[[2238, 2218]]
[[2716, 1858]]
[[2716, 1858]]
[[2210, 1858]]
[[2210, 1858]]
[[3038]]
[[3036, 1852], [3036, 1858]]
[[2237, 1858], [2237, 1852]]
[[2237, 1858], [2237, 1852]]
[[2210, 1858]]
[[2210, 1858]]
[[2716, 2218, 3411]]
[[3285, 1858]]
[[3038, 1813, 1858]]
[[3038, 1813, 1858]]
[[3038, 1813, 1858]]
[[2210, 1858]]
[[3288]]
[[3288]]
[[2210, 1852], [2210, 1858]]
[[2210, 1852], [2210, 1858]]
[[2238]]
[[2238, 2218]]
[[2238, 1858]]
[[3285, 1858]]
[[2716, 1813, 1858]]
[[3288, 1858]]
[[3037]]
[[2210]]
[[2716]]
[[2237, 1858]]
[[2716, 1858]]
[[3288, 1858]]
[[2307, 1858, 3411], [2307, 2218, 3411]]
[[2238, 1858]]
[[2210, 1858]]
[[2210, 1858]]
[[2210, 1858]]
[[3288, 1858]]
[[3038, 1858]]
[[2238]]
[[3265, 2221]]
[[2238, 1858]]
[[2210, 1858]]
[[2237]]
[[2210, 1858], [2210, 2218]]
[[2210, 1858]]
[[2210, 1813, 3411, 1858]]
[[2210, 1813, 3411, 1858]]
[[3038, 1858], [3038, 1852]]
[[3037, 2251], [3037, 1858]]
[[2716, 2218]]
[[3038]]
[[3288, 1858]]
[[3288, 1858]]
[[3038]]
[[2716, 1852], [271

...

[[2210]]
[[2716, 2218, 3411], [2716, 1858, 3411]]
[[2716, 2218, 3411], [2716, 1858, 3411]]
[[2716, 2218, 3411], [2716, 1858, 3411]]
[[2716, 2218, 3411], [2716, 1858, 3411]]
[[2237, 1852]]
[[2237, 1858]]
[[2716]]
[[2716]]
[[2716]]
[[2210, 1852]]
[[2210, 1858]]
[[3265, 2218]]
[[2716]]
[[2716, 1858]]
[[2210, 1852, 3411], [2210, 1854, 3411]]
[[2210, 1852, 3411], [2210, 1854, 3411]]
[[2210]]
[[2210]]
[[2716, 2218]]
[[2307, 1858], [2307, 2218]]
[[2307, 1858], [2307, 2218]]
[[2716, 1858]]
[[2716, 1858]]
[[2307, 1858]]
[[3389, 1858]]
[[3265]]
[[2210, 1858]]
[[2238, 1858]]
[[2238]]
[[3285]]
[[2238]]
[[2716, 1858]]
[[2210, 1858]]
[[2716, 1858], [2716, 2218]]
[[2237]]
[[2716, 1813]]
[[2237]]
[[3370]]
[[3037, 1858]]
[[2237]]
[[2716, 2218], [2716, 1858], [2716, 3069]]
[[2716, 2218, 5143]]
[[3038, 2218]]
[[3265, 1858]]
[[3037, 1858]]
[[3265]]
[[2237, 1858]]
[[2237, 1858]]
[[2237, 1858]]
[[3288]]
[[3288]]
[[2716, 1858]]
[[2307, 1858], [2307, 1852]]
[[3288, 1858]]
[[2238, 1858]]
[[2716, 2218]]
[[2716,

[[2210, 1858]]
[[3288, 1858]]
[[3288, 1858]]
[[2210, 3411]]
[[3390]]
[[3038, 1852]]
[[2238, 1856], [2238, 1858]]
[[3285]]
[[3038, 2234]]
[[3288]]
[[2237, 1858], [2237, 1852]]
[[2236, 1852]]
[[3285, 2218]]
[[2237, 1858], [2237, 2218]]
[[3265, 1858], [3265, 1852]]
[[2716]]
[[3037, 2218]]
[[3285, 1858], [3285, 2218]]
[[3285, 2218]]
[[3285, 1858]]
[[3288, 1858], [3288, 1854], [3288, 3069]]
[[2716]]
[[2210, 2218]]
[[2237, 1858]]
[[3038, 2251]]
[[2236, 1858]]
[[3285]]
[[2307, 2218]]
[[2238, 1858], [2238, 2218]]
[[3390, 1858]]
[[2716, 1858], [2716, 1852]]
[[2210, 2218]]
[[3288, 2251, 3411]]
[[3288, 2251, 3411]]
[[2238, 2218]]
[[3038]]
[[3288]]
[[2716, 1858]]
[[2237, 3069]]
[[2237, 1858]]
[[2716, 1858]]
[[2716, 1858]]
[[2210, 1852]]
[[2210, 3411]]
[[3285]]
[[2307]]
[[2238, 1813, 1858]]
[[2238, 1858]]
[[3285]]
[[3038, 2251], [3038, 1858]]
[[2716, 1858]]
[[3265]]
[[2238, 1852], [2238, 2218], [2238, 1858]]
[[3285]]
[[2210, 1852]]
[[3288, 1858]]
[[2716, 1852], [2716, 1858]]
[[2210, 1858], [2210, 2

[[2238]]
[[2210]]
[[3037, 1852], [3037, 1858]]
[[3037, 1852], [3037, 1858]]
[[2210, 1813, 2208, 1858]]
[[3038]]
[[2210]]
[[3288, 2218]]
[[3038, 1852]]
[[3288, 1858]]
[[3288, 1858, 3411]]
[[3037, 2218]]
[[3036, 1852], [3036, 1856]]
[[3036, 1852], [3036, 1856]]
[[2210, 1858]]
[[2716, 1858]]
[[3038, 1858]]
[[3038, 1858]]
[[2236]]
[[2716, 1858]]
[[2236]]
[[3285, 1858]]
[[2210, 1858, 3411]]
[[2210, 1858, 3411]]
[[3370, 2218]]
[[3370, 2218]]
[[2210]]
[[3265, 1858]]
[[2238, 1858]]
[[2237]]
[[2716]]
[[2237, 1858]]
[[3288, 1856], [3288, 2218]]
[[3285, 1852]]
[[2210, 3411]]
[[3389]]
[[2238, 1858]]
[[3288]]
[[3265]]
[[3038, 1858], [3038, 1852]]
[[2210, 1858]]
[[2210, 1858]]
[[3370]]
[[3370]]
[[3370]]
[[2210, 1852], [2210, 1858]]
[[2716, 1858]]
[[2716, 1856], [2716, 1858]]
[[2716, 2218]]
[[3288, 2218]]
[[3038, 1858, 2208]]
[[3038, 2218], [3038, 1858]]
[[3038]]
[[2237, 1858], [2237, 2218]]
[[3370, 1858]]
[[3038]]
[[3389]]
[[3390, 1858, 3411]]
[[2307, 1858]]
[[3285]]
[[2716, 2251, 3411], [2716, 1858

[[2210, 1858]]
[[2237, 1858], [2237, 1852]]
[[2210]]
[[2210]]
[[2238, 2218]]
[[3265, 1858]]
[[3265, 1858]]
[[3288, 2218]]
[[2210, 2245]]
[[2238, 1852, 3411], [2238, 1858, 3411]]
[[2238, 1858]]
[[2238]]
[[2210, 1858]]
[[2716, 1813, 3411, 1858]]
[[2238, 2218]]
[[2307, 1858], [2307, 1852]]
[[3288, 1838, 2218], [3288, 1838, 1858]]
[[3288]]
[[3265]]
[[3265]]
[[3288, 1858], [3288, 2218]]
[[2238, 1858, 3411], [2238, 1858, 2245]]
[[3038, 1858]]
[[2238, 1858, 2208]]
[[2716]]
[[3037]]
[[3288, 2222, 1858]]
[[2716]]
[[2238]]
[[3288]]
[[2210]]
[[2210]]
[[2716, 1858]]
[[3037, 1858, 2207]]
[[3265]]
[[2238, 2218], [2238, 2251]]
[[2237, 1858]]
[[3288, 1858], [3288, 1852]]
[[2307, 1858]]
[[3288, 1858]]
[[3288, 1858]]
[[2238]]
[[2307, 1852]]
[[2307, 1858]]
[[2716, 1852]]
[[2716]]
[[3038]]
[[2716]]
[[3037, 2218]]
[[2716, 1855, 2245], [2716, 1855, 3411], [2716, 1852, 2245], [2716, 1852, 3411]]
[[2716, 1855, 2245], [2716, 1855, 3411], [2716, 1852, 2245], [2716, 1852, 3411]]
[[2716]]
[[2210, 3411]]
[[2716, 1

[[2236, 3411]]
[[3288, 2218], [3288, 1858]]
[[2716]]
[[3265]]
[[3285, 3411], [3285, 2245]]
[[2210, 1838, 3411]]
[[2237, 1858]]
[[2307]]
[[2716, 1858]]
[[3288, 1858]]
[[2716, 2218], [2716, 1852]]
[[3288, 1858, 3411]]
[[2210, 1858], [2210, 3069]]
[[3038, 1858]]
[[2716, 1858]]
[[2716, 1858]]
[[3037, 1858]]
[[2210, 1858]]
[[2210]]
[[3288, 2218], [3288, 1858]]
[[2307, 1858]]
[[3288, 1858]]
[[2210, 1858], [2210, 2218]]
[[2210]]
[[2210, 1858]]
[[2210, 1858]]
[[2237, 2218]]
[[2716]]
[[3288, 1858]]
[[3288]]
[[2210, 1858], [2210, 2218]]
[[2238, 1858]]
[[3037]]
[[2210, 1858]]
[[3038]]
[[3038, 2218]]
[[2237]]
[[2237]]
[[3037, 1852]]
[[2716, 1858]]
[[2210, 2218]]
[[2210]]
[[3038, 1858]]
[[2716]]
[[3288, 1858], [3288, 1852]]
[[2716, 2218]]
[[3288, 1858]]
[[3288, 1858]]
[[2716]]
[[2210]]
[[3038]]
[[3288, 1813, 1858]]
[[2210, 1852]]
[[3265]]
[[2307, 2218], [2307, 1852]]
[[2716, 1852], [2716, 1858]]
[[2210, 2251]]
[[3038, 1858]]
[[2716, 2218]]
[[2210, 1858]]
[[2210]]
[[2716]]
[[2210, 1858]]
[[2716, 185

In [8]:
print(len(orders))

16561


In [7]:
# here's an example calculation of a pair's support
count = 0
for i in range(len(orders)):
    if 3036 in orders[i] and 1858 in orders[i]:
        count += 1
        
print(count)
print(count/len(orders))


236
0.014250347201255963


In [9]:
# run the apriori algorithm using apyori
# Since we have relativiely few data sets, we only consider pairs {X} -> {Y}
# We limit lift to > 1.05. The min_support and min_confidence levels are chosen
# to get at least a few cases with multiple possible combinations. We then
# pick the one with the greatest product of support * confidence * lift.

association_rules = apriori(orders, min_support=0.0001, min_confidence=0.05, min_lift=1.1, max_length=2)  
assoc = list(association_rules)  

In [10]:
print(len(assoc)) 

131


In [11]:
# messy
for item in assoc:
    if 3036 in list(item.items) and 1858 in list(item.items):
        print(item)

In [12]:
# get products sans Kids and alcohol
df_sa_prods = df_prods.loc[ (df_prods['category'] != 'Kids') & 
                            (df_prods['category'] != 'Alcoholic Beverage'), ['id','name','category']].copy()

df_sa_prods.sort_values('category')

,id,name,category
35,3069,Regular Coffee,Beverage
22,2251,smartwater,Beverage
4,1852,Bottled Water,Beverage
5,1854,Hot Tea,Beverage
6,1855,Chocolate Milk,Beverage
7,1856,Lowfat Milk,Beverage
8,1858,Fountain Beverage,Beverage
12,2218,Strawberry Lemonade Slushy,Beverage
36,3070,Decaf Coffee,Beverage
21,2245,Snowman Cupcake,Dessert


In [13]:
# build the recommendation dict of dicts for Entrees
prodlist = df_sa_prods.T.to_dict().values()

reco_dict = {}
cats = ['Entree','Side','Dessert','Beverage']


for prod in prodlist:
    print(prod)
    md = {}
    id = prod['id']
    md['category'] = prod['category']
    md[prod['category']] = -1
    md['Kids'] = -1

    search_cats = [ x for x in cats if x != prod['category'] ]
    print(search_cats)
    print(id)
    
    dd = [ x for x in assoc if list(x.items)[0] == id or list(x.items)[1] == id  ]
    # for each category, we take the pairing with the highest
    # support, since we have already placed a good limit on lift
    # If there is no pairing, we set that category to -2
    for cat in search_cats:
        
        xl = []
        for x in dd:
            for y in x.ordered_statistics:
                xd = {}
                xid = list(y.items_add)[0]
                if xid != id:
                    xd['id'] = xid
                    xd['support'] = x.support
                    xd['confidence'] = y.confidence
                    xd['lift'] = y.lift
                    if get_cat(xid) == cat:
                        xl.append(xd)    


        
        xs = sorted(xl,key = lambda x: x['support']*x['confidence']*x['lift'],reverse=True)
        if len(xs) > 0:
            md[cat] = xs[0]
        else:
            md[cat] = -2
        print(cat,xs)
        
        print('**')

    print('------------------')
        
    
    
    reco_dict[prod['id']] = md
    
    
    

{'id': 1813, 'name': 'French Fries', 'category': 'Side'}
['Entree', 'Dessert', 'Beverage']
1813
Entree [{'id': 2716, 'support': 0.005736368576776765, 'confidence': 0.2074235807860262, 'lift': 1.1664318918157488}, {'id': 2237, 'support': 0.00446832920717348, 'confidence': 0.1615720524017467, 'lift': 1.8018819931483685}, {'id': 2307, 'support': 0.002475695912082604, 'confidence': 0.08951965065502183, 'lift': 1.1850798836913004}]
**
Dessert [{'id': 3411, 'support': 0.0015699535052231144, 'confidence': 0.05676855895196506, 'lift': 1.260246789280822}]
**
Beverage [{'id': 1858, 'support': 0.015216472435239417, 'confidence': 0.5502183406113537, 'lift': 1.1680766490020034}, {'id': 1852, 'support': 0.003321055491818127, 'confidence': 0.12008733624454149, 'lift': 1.1823819117395074}]
**
------------------
{'id': 1838, 'name': 'Green Beans', 'category': 'Side'}
['Entree', 'Dessert', 'Beverage']
1838
Entree [{'id': 3288, 'support': 0.001147273715355353, 'confidence': 0.152, 'lift': 1.3548288482238

In [14]:
print(json.dumps(reco_dict,indent=2))

{
  "1813": {
    "category": "Side",
    "Side": -1,
    "Kids": -1,
    "Entree": {
      "id": 2716,
      "support": 0.005736368576776765,
      "confidence": 0.2074235807860262,
      "lift": 1.1664318918157488
    },
    "Dessert": {
      "id": 3411,
      "support": 0.0015699535052231144,
      "confidence": 0.05676855895196506,
      "lift": 1.260246789280822
    },
    "Beverage": {
      "id": 1858,
      "support": 0.015216472435239417,
      "confidence": 0.5502183406113537,
      "lift": 1.1680766490020034
    }
  },
  "1838": {
    "category": "Side",
    "Side": -1,
    "Kids": -1,
    "Entree": {
      "id": 3288,
      "support": 0.001147273715355353,
      "confidence": 0.152,
      "lift": 1.3548288482238966
    },
    "Dessert": {
      "id": 3411,
      "support": 0.0006038282712396595,
      "confidence": 0.08,
      "lift": 1.7759785522788203
    },
    "Beverage": {
      "id": 2218,
      "support": 0.0009661252339834551,
      "confidence": 0.128,
      "lift

In [15]:
# re-key dictionary so that it fits with what we have
# in the flask app
for key in reco_dict.keys():
    xlist = list(reco_dict[key].keys())
    for xkey in xlist:
        xx = reco_dict[key][xkey]
        if xkey == 'Alcoholic Beverage':
            reco_dict[key]['alcohol'] = xx
            del reco_dict[key][xkey]
        else:
            new_key = xkey.lower()
            reco_dict[key][new_key] = xx
            del reco_dict[key][xkey]


In [16]:
with open("../data/teens_recommendations.json",'w') as f:
    f.write(json.dumps(reco_dict,indent=2))